<h1 style="font-size: 36px; color: #FFD700">FINE-TUNING</h1>

`1`. `Fine-tuning` 

`Finetune` - điều chỉnh mô hình được đào tạo trước cho một một tác vụ cụ thế với một tạp dữ liệu chuyên biệt nhỏ hơn.
- Các tiếp cận này đòi hỏi ít dữ liệu và tính toán hơn so với việc đào tạo mô hình từ đầu

=> Tạo thành cách tiếp cận dễ hơn với nhiều người dùng.

`Transformer` cung cấp `Trainer API`, cung cấp bộ tính năng đào tạo toàn diện để finetune bất kỳ mô hình nào trên Hub

Ta sẽ sử dụng tập `Yelp Review` và xử lý (phân loại `tokenize`, thêm `pad`, cắt `truncate`) cho việc training.
- Dùng `map` để preprocessing toàn bộ tập dữ liệu trong một bước

*`tokenizer` - là bộ xử lý văn bản
- Chứa các thành phần cố định như 
    - Vocabulary - ánh xạ từ con (tokens) sang ID số
    - `tokenization rules` - quy tắc tách từ
    - Các tham số cấu hình xử lý như độ dài tối đa, cách xử lý từ không có trong vocab
- tokenizer đóng vai trò `phiên dịch` giữa văn bản thật và `token` - ngôn ngữ mà models hiểu được.  

*`token` - là đơn vị nhỏ nhất mà mô hình ngôn ngữ có thể xử lý, tương tự như pixel của ảnh, số nguyên tố trong toán.

Các loại token phổ biến:
- `Word Token`

    - `Mỗi từ là 1 token riêng biệt`
    - Vd: "ChatGPT rất mạnh" => ["ChatGPT", "rất", "mạnh"]

- `Subword Token`

    - `Tách từ thành các phần nhỏ hơn`
    - Vd: "ChatGPT" => ["Chat", "G", "PT"]

- `Character Token`
    
    - `Mỗi kí tự là 1 token`
    - Vd: "NLP" => ["N", "L", "P"]

Quá trình token hóa thực tế
- Token hóa cơ bản => ["Tôi", "thích", "ăn", "phở", "!"]
- Token hóa từ con (WordPiece) =>  ["Tôi", "thích", "ăn", "ph", "##ở", "!"]
- Chuyển thành ID số => [1045, 2214, 2514, 6421, 4789, 999]

Đặc điểm kỹ thuật của token
- Từ vựng token (Vocabulary): Tập hợp tất cả token mà mô hình biết
- Kích thước từ vựng: Thường từ 30k-100k token với mô hình lớn
- Ánh xạ 2 chiều: Token ↔ Token ID (số nguyên)


In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("yelp_review_full") # Tải dataset "yelp_review_full"
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased") 
# Tải tokenizer đã được huấn luyện trước từ BERT

def tokenize(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128) # Giảm max_length để tránh tràn bộ nhớ

dataset = dataset.map(tokenize, batched=True)

/home/qk/Documents/python/python_programming/pytorch/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 50000/50000 [00:03<00:00, 13256.20 examples/s]


`Giải thích`

`tokenize` - thực hiện chuyển đổi văn bản thành các token bằng cách dùng `Hàm tokenizer`
- các đoạn văn bản sẽ được cắt ngắn nếu quá dài (`truncation=True`)
- (`padding="max_length"`) để đạt được độ dài tối đa cho mỗi chuỗi token
    - Minh họa:
    
        - `Text 1`: "I love programming."
        - `Text 2`: "Python is a powerful programming language."

    - `Truncation`:

        - `Text 1`: ["I", "love", "programming", ".", "[SEP]", "[CLS]"] - vì độ dài text = 4 nên khi max_length=5 thì nó không cần phải bỏ bớt từ nào hết.
        - `Text 2`: ["Python", "is", "a", "powerful", "programming"] - text dài hơn 5 nên nó bị cắt thành 5 token
        => Nếu văn bản quá dài, token sẽ bị loại bỏ từ cuối để chỉ giữ lại số lượng token tối đa.

    - `Padding`:
    
        - Giả sử chúng ta có một max_length=5 và padding="max_length", tokenizer sẽ đệm các token thiếu bằng ký tự đặc biệt [PAD] để đạt độ dài tối đa
        - Ví dụ:
            - `text 1`: Sau khi padding, nó sẽ trở thành: ["I", "love", "programming", ".", "[PAD]"]
            - `text 2`: Sau khi padding, không cần thêm gì vì nó đã có đủ 5 token.

=> `truncation` giúp cắt bớt văn bản quá dài, và `padding` giúp đảm bảo rằng mọi đoạn văn bản đều có cùng độ dài

`.map` - áp dụng hàm tokenize cho mỗi phần tử trong dataset, `batched=True` sẽ giúp xử lý các ví dụ theo nhóm => giúp tăng tốc quá trình 

Tinh chỉnh trên một phần nhỏ của toàn bộ tập dữ liệu giúp giảm thời gian thực hiện. Kết quả có thể không tốt bằng khi tinh chỉnh trên toàn bộ tập dữ liệu, nhưng cách này sẽ giúp bạn đảm bảo mọi thứ hoạt động đúng trước khi quyết định đào tạo trên toàn bộ dữ liệu.

=> đây là phép thử để kiểm tra cách hoạt động của models có giống với kì vọng không mà không làm tốn quá nhiều thời gian.

In [2]:
small_train = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval = dataset["test"].shuffle(seed=42).select(range(1000))

`2`. `Trainer`

Là một vòng lặp đào tạo được tối ưu hóa cho các mô hình Transformers, giúp chúng ta dễ dàng bắt đầu đào tạo ngay mà không cần phải tự viết mã đào tạo lại.
- Chọn và lựa chọn từ nhiều tính năng đào tạo trong TrainingArguments như tích lũy gradient, độ chính xác hỗn hợp và các tùy chọn để báo cáo và ghi nhật ký số liệu đào tạo.

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)
"Một số trọng số của BertForSequenceClassification không được khởi tạo từ điểm kiểm tra mô hình tại google-bert/bert-base-cased và được khởi tạo mới: ['classifier.bias', 'classifier.weight']" 
"Bạn có thể nên HUẤN LUYỆN mô hình này trên một tác vụ hạ nguồn để có thể sử dụng nó cho các dự đoán và suy luận."

2025-04-11 01:07:27.314480: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 01:07:27.323254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744308447.333900   70899 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744308447.337051   70899 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744308447.345509   70899 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

'Bạn có thể nên HUẤN LUYỆN mô hình này trên một tác vụ hạ nguồn để có thể sử dụng nó cho các dự đoán và suy luận.'

`num_labels=5` sẽ thay đổi lớp cuối cùng thành tác vụ phân loại với 5 nhãn khác nhau.

*`Note`:Thông báo ở trên là lời nhắc nhở rằng đầu được đào tạo trước của mô hình bị loại bỏ và thay thế bằng đầu phân loại được khởi tạo ngẫu nhiên. Đầu được khởi tạo ngẫu nhiên cần được tinh chỉnh trên tác vụ cụ thể của bạn để đưa ra các dự đoán có ý nghĩa.

Với mô hình đã tải, ta có thể thiết lập `hyperparameter` trong TrainingArguments. 
- `Hyperparameter` - là các biến kiểm soát quá trình traning (lr, size_batch, epoch) từ đó ảnh hưởng đến hiệu suất của mô hình. 
- Việc chọn hyperparameters rất quan trọng.

`Trainer` yêu cầu một hàm để tính toán và báo cáo số liệu. Đối với tác vụ phân loại, ta sẽ dùng evaluate.load để tải hàm `Evaluate` từ thư viện Evaluate. Sau đó thu thập các dự đoán và nhãn compute để tính toán evaluate

In [4]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy") # lệnh tải metrics accuracy

def compute_metrics(eval_pred):
    logits, labels = eval_pred # tuple chứa logits và labels
    
    predictions = np.argmax(logits, axis=-1) # Lấy giá trị lớn nhất trong logits dọc theo chiều cuối cùng
    return metric.compute(predictions=predictions, references=labels) # sau khi có pred thì nó sẽ tính acc bằng cách so sánh chúng với nhãn thực tế

In [7]:
# Thiết lập TrainingArguments với nơi save models và thời điểm tính acc
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="yelp_review_classifier", # thư mục chứa model
    eval_strategy="epoch", # thời gian tính accuracy
    push_to_hub=True, # tải models lên Hub sau khi train
    # Thêm 2 dòng dưới vào để giảm batch size để không bị tràn bộ nhớ
    per_device_train_batch_size=8,  # Giảm kích thước batch size
    per_device_eval_batch_size=8,   # Giảm kích thước batch size cho eval
)

In [8]:
from transformers import Trainer
# Khởi tạo một Trainer và truyền cho nó model, đối số đào tạo, tập train, test
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics, 
)
trainer.train() # gọi đào tạo

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

Hiện tại tôi đang áp dụng 2 cách để giảm thiểu việc sử dụng bộ nhớ GPU và tránh lỗi OutOfMemoryError
- Tôi giảm kích thước batch size => giảm tải bộ nhớ GPU
- Giảm độ dài chuỗi đầu vào (sequence length)

`TensorFlow`

Trainer không tương thích với các mô hình Trainsformers TensorFlow nên ta thay bằng Keras vì các mô hình Transformer được triển khai dưới dạng `tf.keras.Model`

In [ ]:
# from transformers import TFAutoModelForSequenceClassification
# from datasets import load_dataset
# from transformers import AutoTokenizer

# model = TFAutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)
# dataset = load_dataset("yelp_review_full")
# tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

# def tokenize(examples):
#     return tokenizer(examples["text"])

# dataset = dataset.map(tokenize)

`tf.data.Dataset` - một lớp trong TensorFlow (tf) giúp xử lý và quản lý dữ liệu hiệu quả khi huấn luyện mô hình học sâu. Nó cung cấp các công cụ để tạo ra các pipeline dữ liệu (data pipeline) có thể lặp lại, dễ dàng phân tán, và có thể tối ưu hóa cho việc huấn luyện mô hình. `datapipeline` này chủ yếu để transform data trước khi đưa vào dữ liệu huấn luyện

Có hai phương pháp để chuyển đổi một tập dữ liệu thành `tf.data.Dataset`

- `prepare_tf_dataset()` là cách được khuyến nghị để tạo tf.data.Dataset vì ta có thể kiểm tra mô hình để tìm ra cột nào sẽ sử dụng làm đầu vào và cột nào sẽ loại bỏ => tạo ra dataset đơn giản, hiệu suất cao

- `to_tf_dataset` là phương thức cấp thấp hơn trong thư viện Datasets cho phép kiểm soát tốt hơn cách tạo tập dữ liệu bằng cách chỉ định các cột và nhãn cột cần sử dụng.

Thêm `tokenize` vào để `prepare_tf_dataset()` để padding cho từng batch

Trong TensorFlow và PyTorch, việc xử lý dữ liệu (như tokenization, padding, augmentation, v.v.) có thể được thực hiện qua một hàm `collate_fn` (trong PyTorch).

Trong PyTorch, `collate_fn` là một hàm tuỳ chỉnh dùng để kết hợp các phần tử của một batch và xử lý chúng trước khi truyền vào mô hình.
- Hàm này rất hữu ích khi bạn cần thực hiện những bước xử lý phức tạp như padding không đều hoặc xử lý dữ liệu đặc biệt trước khi đưa vào mô hình
- `Uneven padding` - Là việc sử dụng padding linh hoạt cho mỗi chuỗi, sao cho mỗi chuỗi được đệm đến độ dài tối đa của chính nó thay vì một giá trị cố định.

In [ ]:
# tf_dataset = model.prepare_tf_dataset(
#     dataset["train"], batch_size=16, shuffle=True, tokenizer=tokenizer
# )

In [ ]:
# # Biên dịch và điều chỉnh để bắt đầu train
# from tensorflow.keras.optimizers import Adam

# model.compile(optimizer=Adam(3e-5))
# model.fit(tf_dataset)